In [ ]:
# Import Libraries
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization
import tensorflow as tf

import cv2
import numpy as np

In [ ]:
# Model parameters
WIDTH = 150
HEIGHT = 150
LR = 1e-3
EPOCHS = 6

In [ ]:
# labels (for images)
a = [1,0,0]
b = [0,1,0]
c = [0,0,1]

In [ ]:
def one_hot_to_label(one_hot):
    if one_hot == a:
        return 'Fist'
    if one_hot == b:
        return 'OpenHand'
    if one_hot == c:
        return 'TwoFinger'

In [ ]:
def alexnet(width, height, lr):
    network = input_data(shape=[None, width, height, 1], name='input')
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 3, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=lr, name='targets')

    model = tflearn.DNN(network, checkpoint_path='model_alexnet',
                        max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir='log')
    return model

In [ ]:
tf.reset_default_graph()

In [ ]:
model = alexnet(WIDTH, HEIGHT, LR)

In [ ]:
model.load('./model_ver10')

In [ ]:
cap = cv2.VideoCapture(0)
while cv2.waitKey(1):

    _, frame = cap.read()                      
    blur = cv2.GaussianBlur(frame, (5,5), 0)
    
    hsv = cv2.cvtColor(blur, cv2.COLOR_RGB2HSV)
    
    lower_skin = np.array([110, 25, 85])
    upper_skin = np.array([179, 255, 255])
    
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    blur = cv2.medianBlur(mask, 15)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 8))
    hsv_d = cv2.dilate(blur, kernel)

    cv2.rectangle(frame, (0, 0), (300, 300), (255, 0, 0), 2)
    
    final = cv2.resize(hsv_d[:300, :300], (150, 150))
    one_hot = list(np.around(model.predict([final.reshape(150, 150, 1)])[0]))
    prediction = one_hot_to_label(one_hot)
    cv2.putText(hsv_d,prediction, (5,25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.imshow('Original', frame)
    cv2.imshow('Mask', hsv_d[:300, :300])
    
cv2.destroyAllWindows()
cap.release()